In [21]:
import datetime as dt
import csv
import pandas as pd
import networkx as nx

In [12]:
vaccine_terms = 'shot|vaccine|vacine|vacines|vaccines|vaccinate|vaccination|vaccinations|vaccinated|vaccinating|vaxxed|vaxx|vax|\
unvaccinated|unvaxxed|antivaxx|antivaccination|anti|\
moderna|pfizer|J&J|immune|immunize|immunizes|immunized|immunization|immunizations'
vaccine_terms_list= vaccine_terms.split('|')
vaccine_terms_list

['shot',
 'vaccine',
 'vacine',
 'vacines',
 'vaccines',
 'vaccinate',
 'vaccination',
 'vaccinations',
 'vaccinated',
 'vaccinating',
 'vaxxed',
 'vaxx',
 'vax',
 'unvaccinated',
 'unvaxxed',
 'antivaxx',
 'antivaccination',
 'anti',
 'moderna',
 'pfizer',
 'J&J',
 'immune',
 'immunize',
 'immunizes',
 'immunized',
 'immunization',
 'immunizations']

In [ ]:
# use your own path for the data
data_path = '/Volumes/GoogleDrive-105418601834042090131/My Drive/math168_project/data'

In [ ]:
import os
import glob
  
csv_files = glob.glob(os.path.join(data_path, "*.csv"))

for f in csv_files:
      
    # read the csv file
    df = pd.read_csv(f)
      
    # print the location and filename
    df = df[['user_id', 'text', 'created_at', 'id', 'in_reply_to_screen_name', 'retweet_screen_name', 'user_screen_name']]
    df = df.dropna(subset=['in_reply_to_screen_name', 'retweet_screen_name'], how='all')
    print(len(df))
    df = df[df.text.str.contains(vaccine_terms)]
    print(len(df))
    G = nx.from_pandas_edgelist(df, 'user_screen_name', 'in_reply_to_screen_name', create_using=nx.MultiGraph())
    nx.draw(G)
    break



/var/folders/1w/qqx3tnc943383tv49dm69fd00000gn/T/ipykernel_35662/330679818.py:9: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


350425
112895
